In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

In [11]:
 df = pd.read_csv("training_data.csv")

In [12]:
 X = df.drop("target", axis=1)
 y = df["target"]


In [13]:
 obj_cols = X.select_dtypes(include="object").columns
 X = X.drop(columns=obj_cols)


In [14]:
 from sklearn.impute import SimpleImputer

 imputer = SimpleImputer(strategy="median")
 X[:] = imputer.fit_transform(X)


In [15]:
 from sklearn.feature_selection import mutual_info_classif

 mi = mutual_info_classif(X, y, random_state=42)
 mi = pd.Series(mi, index=X.columns).sort_values(ascending=False)

 top_features = mi.head(30).index   # tune this number
 X = X[top_features]


In [7]:
# from sklearn.model_selection import StratifiedKFold

# skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


In [8]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score

# auc_scores = []

# for train_idx, val_idx in skf.split(X, y):

#     X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

#     scaler = StandardScaler()
#     X_tr = scaler.fit_transform(X_tr)
#     X_va = scaler.transform(X_va)

#     model = LogisticRegression(
#         class_weight="balanced",
#         max_iter=1000
#     )

#     model.fit(X_tr, y_tr)

#     preds = model.predict_proba(X_va)[:, 1]
#     auc_scores.append(roc_auc_score(y_va, preds))


In [9]:
# mean_auc = np.mean(auc_scores)
# gini = 2 * mean_auc - 1

# print("AUC:", mean_auc)
# print("Normalized Gini:", gini)


AUC: 0.6025202678977972
Normalized Gini: 0.20504053579559445


In [16]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

X = X_train.copy()
y = y_train.copy()

neg = (y == 0).sum()
pos = (y == 1).sum()
scale_pos_weight = neg / pos

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y)):
    print(f"\nFold {fold+1}")

    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    model = CatBoostClassifier(
        iterations=3000,
        depth=8,
        learning_rate=0.03,
        loss_function="Logloss",
        eval_metric="AUC",
        scale_pos_weight=scale_pos_weight,
        random_seed=fold * 13,
        verbose=200,
        early_stopping_rounds=200
    )

    model.fit(
        X_tr, y_tr,
        eval_set=(X_va, y_va),
        use_best_model=True
    )

    oof_preds[va_idx] = model.predict_proba(X_va)[:, 1]
    test_preds += model.predict_proba(X_test)[:, 1] / skf.n_splits


NameError: name 'X_train' is not defined

In [ ]:
auc = roc_auc_score(y, oof_preds)
gini = 2 * auc - 1

print("OOF AUC:", auc)
print("OOF Gini:", gini)
